In [1]:
import numpy as np
import json
import os
import re
import collections
from sklearn.datasets.base import load_files
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import codecs
import pickle
from gensim.models.word2vec import LineSentence
from gensim.models.fasttext import FastText
import sys
from __future__ import division
from collections import OrderedDict
import pandas as pd
from sklearn.datasets import load_files
from collections import OrderedDict

In [2]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.model_selection import KFold
from surprise.prediction_algorithms import KNNBasic
from surprise import accuracy
from threading import Thread
from surprise.model_selection import train_test_split

In [3]:
def normalize_name(name):
    name = name.lower()
    name = re.sub(r"[.,\/#!$%\^\*;:{}=\_`~()@]", ' ', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

In [4]:
#Load Training Data change path name as per your directory
path = '/home/irlab/Documents/recsys/mpd/data'
mapping = {}
playlist_prob = OrderedDict()

In [5]:
#Mapping of playlist name to filename
def load_filemapping():
    fp = open('/home/irlab/files/mappings/track_filename.json')
    mpd_mapping = json.load(fp)
    return mpd_mapping

In [6]:
mpd_mapping = load_filemapping()

In [7]:
#Change this as per you files folder location
fp = open('/home/irlab/files/stats/album_stats.json')
album_map = json.load(fp)
album_stat = sum(album_map.values())

#Change this as per you files folder location
fp = open('/home/irlab/files/stats/artist_stats.json')
artist_map = json.load(fp)
artist_stat = sum(artist_map.values())

#Change this as per you files folder location
fp = open('/home/irlab/files/stats/tracks_stats.json')
track_map = json.load(fp)
track_stat = sum(track_map.values())

print(album_stat,artist_stat,track_stat)

(66346428, 66346428, 66346428)


In [8]:
#Get playlist name, track and value
def data_dict():
    for key,value in playlist_prob.items():
        for subkey,subvalue in value.items():
            yield key,subkey,subvalue

In [9]:
#Load Data in surprise format
def load_data():
    df = pd.DataFrame(data=data_dict(),columns=['userID','itemID','rating'])
    reader = Reader(rating_scale=(0,1))
    data = Dataset.load_from_df(df[['userID','itemID','rating']],reader)
    n_tracks = df['itemID']
    yield data,n_tracks

In [10]:
#Training has been performed using Collaborative Filering
#For similarity cosine similarity has been used and similarity between users is calculated

def train(data):
    
    sim_options = { 'name' : 'cosine',
                    'user_based' : True
                  }
    def data_send(data):
        yield data
        
    trainset, testset = train_test_split(data_send(data).next(),test_size=.25)
    
    def train():
        yield trainset
    def test():
        yield testset
    
    algo = KNNBasic(k=40,sim_options=sim_options)
    algo.fit(next(train()))
    prediction = algo.test(next(test()))
    print(accuracy.rmse(prediction,verbose=True))
    return algo

In [11]:
#Predict ratings for empty tracks

def test(playlist_name,tracks,algo,seed_tracks):
    def track_filter():
        for track in tracks:
            if track not in seed_tracks:
                #print(track)
                yield track
    
    def track_predict(tracks_pred):
            yield tracks_pred
        
    def sort_dict(Recommend):
        for (key,value) in sorted(Recommend.items(),key=lambda x:x[1],reverse=True):
            yield key
        
    print('Filtering Tracks...')
    tracks_pred = [track for track in track_filter()]
    Recommend = {}
    print('Predictng...')
    for track in next(track_predict(tracks_pred)):
        predict = algo.predict(playlist_name,track)
        Recommend[track] = predict[3]
    print('Sorting List')
    recommend_list = [key for key in sort_dict(Recommend)]
    print('Sending List')
    return recommend_list[0:500]

In [12]:
#Load data with scores
fp = open('/home/irlab/files/mappings/track_scores.json')
playlist_prob = json.load(fp)

In [13]:
#Change the directory as per your directories

path1 = '/home/irlab/Documents/recsys/challenge'
challenge_file = 'challenge_set.json'
filename = os.sep.join((path1,challenge_file))
fp = open(filename)
challenge_set = json.load(fp)
playlists = challenge_set['playlists']

In [14]:
import csv

In [15]:
#Load exisiting ratings into the dicitonary
count = 0

test_tracks = {}

for playlist in playlists:
    if 'name' not in playlist.keys() and playlist['num_samples'] == 10:
        count+=1
        print(count,playlist['pid'])
        playlist_prob[playlist['pid']] = {}
        seed_tracks = []
        i = 0
        list_temp = []
        list_temp.insert(0,playlist['pid'])
        for track in playlist['tracks']:
            p1 = artist_map[track['artist_uri']]/artist_stat
            p2 = album_map[track['album_uri']]/album_stat
            p3 = track_map[track['track_uri']]/track_stat
            playlist_prob[playlist['pid']][track['track_uri']] = (p1+p2+p3)/(1+track['pos'])
            seed_tracks.insert(i,track['track_uri'])
        test_tracks[playlist['pid']] = seed_tracks

(1, 1010764)
(2, 1010773)
(3, 1010774)
(4, 1010775)
(5, 1010780)
(6, 1010782)
(7, 1010786)
(8, 1010791)
(9, 1010796)
(10, 1010799)
(11, 1010800)
(12, 1010803)
(13, 1010806)
(14, 1010807)
(15, 1010809)
(16, 1010813)
(17, 1010819)
(18, 1010820)
(19, 1010827)
(20, 1010828)
(21, 1010830)
(22, 1010836)
(23, 1010837)
(24, 1010838)
(25, 1010840)
(26, 1010843)
(27, 1010845)
(28, 1010849)
(29, 1010856)
(30, 1010862)
(31, 1010863)
(32, 1010869)
(33, 1010870)
(34, 1010877)
(35, 1010881)
(36, 1010883)
(37, 1010884)
(38, 1010886)
(39, 1010889)
(40, 1010897)
(41, 1010898)
(42, 1010907)
(43, 1010910)
(44, 1010911)
(45, 1010912)
(46, 1010917)
(47, 1010919)
(48, 1010923)
(49, 1010928)
(50, 1010930)
(51, 1010932)
(52, 1010935)
(53, 1010940)
(54, 1010942)
(55, 1010946)
(56, 1010951)
(57, 1010953)
(58, 1010957)
(59, 1010961)
(60, 1010963)
(61, 1010965)
(62, 1010971)
(63, 1010972)
(64, 1010977)
(65, 1010980)
(66, 1010981)
(67, 1010983)
(68, 1010985)
(69, 1010987)
(70, 1010988)
(71, 1010994)
(72, 1010996)
(

In [16]:
#Send data for training
print('Loading data for training')
data,n_tracks = next(load_data())
print('Training....')
algo = train(data)

Loading data for training
Training....
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.0004
0.00035357771308093094


In [17]:
#Send Tracks with missing values to predict ratings

count = 0
for playlist_name in test_tracks.keys():
    i = 0
    count+=1
    print(count)
    seed_tracks = []
    list_temp = []
    list_temp.insert(0,playlist_name)
    seed_tracks = test_tracks[playlist_name]
    recommend_list = test(playlist_name,n_tracks,algo,seed_tracks)
    #print(recommend_list)
    list_temp.extend(recommend_list)
    print(len(recommend_list))
    print('Writing to file.....')
    #Change this as per your directory
    with open('/home/irlab/Documents/recsys/mpd/codes/output/final_submission.csv','a') as csvfile1:
        filewriter = csv.writer(csvfile1,delimiter = ',')
        filewriter.writerow(list_temp)

1
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
2
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
3
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
4
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
5
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
6
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
7
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
8
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
9
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
10
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
11
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
12
Filtering Tracks...
Predictng...
Sorting List
Sen

Sorting List
Sending List
500
Writing to file.....
96
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
97
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
98
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
99
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
100
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
101
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
102
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
103
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
104
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
105
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
106
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writin

Sending List
500
Writing to file.....
189
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
190
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
191
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
192
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
193
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
194
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
195
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
196
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
197
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
198
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
199
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file

Predictng...
Sorting List
Sending List
500
Writing to file.....
283
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
284
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
285
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
286
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
287
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
288
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
289
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
290
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
291
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
292
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
293
Filtering Tracks...
Predictng...
Sorting List
Sendin

Sorting List
Sending List
500
Writing to file.....
376
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
377
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
378
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
379
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
380
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
381
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
382
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
383
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
384
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
385
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
386
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Wr

...

Predictng...
Sorting List
Sending List
500
Writing to file.....
563
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
564
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
565
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
566
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
567
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
568
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
569
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
570
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
571
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
572
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
573
Filtering Tracks...
Predictng...
Sorting List
Sendin

Sorting List
Sending List
500
Writing to file.....
656
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
657
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
658
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
659
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
660
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
661
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
662
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
663
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
664
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
665
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
666
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Wr

Sending List
500
Writing to file.....
749
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
750
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
751
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
752
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
753
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
754
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
755
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
756
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
757
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
758
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
759
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file

Predictng...
Sorting List
Sending List
500
Writing to file.....
843
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
844
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
845
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
846
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
847
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
848
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
849
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
850
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
851
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
852
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
853
Filtering Tracks...
Predictng...
Sorting List
Sendin

Sorting List
Sending List
500
Writing to file.....
936
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
937
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
938
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
939
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
940
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
941
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
942
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
943
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
944
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
945
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Writing to file.....
946
Filtering Tracks...
Predictng...
Sorting List
Sending List
500
Wr